In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torch.utils.data as Data
import matplotlib.pyplot as plt
import seaborn as sns
import os
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

 
process_dataset_x= pd.read_csv('/home/ysy/ysy/Fed-ReKD-dirs/process_dataset_copy_x.csv',index_col=0)
process_dataset_y= pd.read_csv('/home/ysy/ysy/Fed-ReKD-dirs/process_dataset_copy_y.csv',index_col=0)
x = process_dataset_x
y = process_dataset_y['logerror']
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.1,random_state=42)
 
scale=StandardScaler()
train_x=scale.fit_transform(train_x)
test_x=scale.fit_transform(test_x)
 
del x,y
train_x.shape,train_y.shape,test_x.shape,test_y.shape,max(train_y),min(train_y),max(test_y),min(test_y)

((79637, 81), (79637,), (8849, 81), (8849,), 0.4134, -0.3901, 0.4121, -0.3857)

In [2]:
train_x=np.array(train_x)
train_y=np.array(train_y)
test_x=np.array(test_x)
test_y=np.array(test_y)

In [3]:
# Split dataset
def split_list_n_list(origin_list, n):
    if len(origin_list) % n == 0:
        cnt = len(origin_list) // n
    else:
        cnt = len(origin_list) // n + 1
 
    for i in range(0, n):
        yield origin_list[i*cnt:(i+1)*cnt]

In [4]:
#preparing teacher's datasets
n_teachers=5

teacher_x,teacher_y = [],[]
teacher_datasets = []
teacher_data_loader = []



teacher_x_loder = split_list_n_list(train_x,n_teachers)
teacher_y_loder = split_list_n_list(train_y,n_teachers)

teacher_x.extend(iter(teacher_x_loder))
teacher_y.extend(iter(teacher_y_loder))

for i in range(n_teachers):
    print(i,teacher_x[i].shape,teacher_y[i].shape)


from torch.utils.data import TensorDataset, DataLoader
teacher_datasets.extend(TensorDataset(torch.tensor(teacher_x[u],device=device,dtype=torch.float),
                                       torch.tensor(teacher_y[u],device=device,dtype=torch.float))
                        for u in  range(n_teachers))

teacher_data_loader.extend(DataLoader(teacher_datasets[i],batch_size=128,shuffle=False)
                           for i in range(n_teachers))


student_x,student_y = [] ,[]
student_datasets= []
 

student_x_loder =  split_list_n_list(test_x,2)
student_y_loder =  split_list_n_list(test_y,2)
student_x.extend(iter(student_x_loder))
student_y.extend(iter(student_y_loder))

for i in range(2):
    print(i,student_x[i].shape,student_y[i].shape)

student_datasets.extend(
                        TensorDataset(torch.tensor(student_x[u],device=device,dtype=torch.float),
                                      torch.tensor(student_y[u],device=device,dtype=torch.float))
                        for u in range(2)
)

student_train_loader = DataLoader(student_datasets[0], batch_size=len(student_datasets[0]),shuffle=True)
student_test_loader = DataLoader(student_datasets[1], batch_size=len(student_datasets[1]),shuffle=True)

0 (15928, 81) (15928,)
1 (15928, 81) (15928,)
2 (15928, 81) (15928,)
3 (15928, 81) (15928,)
4 (15925, 81) (15925,)
0 (4425, 81) (4425,)
1 (4424, 81) (4424,)


In [5]:
from random import shuffle
from torch.utils.data import TensorDataset, DataLoader
 
from opacus.utils.uniform_sampler import UniformWithReplacementSampler

train_x=np.array(train_x)
train_y=np.array(train_y)
test_x=np.array(test_x)
test_y=np.array(test_y)


train_dataset=TensorDataset(torch.tensor(train_x,dtype=torch.float),torch.tensor(train_y,dtype=torch.float))
train_loader=DataLoader(train_dataset,num_workers=1, batch_size=128,shuffle=True)

test_dataset=TensorDataset(torch.tensor(test_x,dtype=torch.float)
                    ,torch.tensor(test_y,dtype=torch.float))
test_loader=DataLoader(train_dataset,num_workers=1, batch_size=128,shuffle=True)
len(train_dataset)

In [6]:
from pickletools import optimize


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden1 = nn.Sequential(nn.Linear(in_features=81, out_features=810, bias=True),
                                  nn.Dropout(0.7),
                                  nn.Tanh()
                                  )
        
        self.hidden2 = nn.Sequential(nn.Linear(in_features=810, out_features=162, bias=True),
                                  nn.Dropout(0.7),
                                  nn.Tanh()
                                  )
        
        self.hidden3 = nn.Sequential(nn.Linear(in_features=162, out_features=54, bias=True),
                                  nn.Dropout(0.6),
                                  nn.Tanh()
                                  )
        
        self.predict = nn.Sequential(nn.Linear(in_features=54, out_features=1, bias=True),
                                        nn.Tanh())
        
    def forward(self, x):
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        output = self.predict(x)
        return output



model=MLP().to(device)
teachers_models = []
import torch.nn as nn
import torch.optim as optim
for i in range(n_teachers):
    net = model
    teachers_models.append(net)
 
teacher_optimizers = [torch.optim.SGD(teachers_models[i].parameters(), lr=1e-3) for i in range(n_teachers)]
#creating teachers folders
def  mkdir_if_missing(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

for i in  range(n_teachers):
    mkdir_if_missing(f'/home/ysy/ysy/Fed-ReKD-dirs/MLP_DP_SGD/teacher{i}')


In [7]:
 
#training configs
num_epochs=20  #10*3000epoch
batch_size =128
lr =1e-4

In [8]:
#training teacher models
loss_func=nn.MSELoss()
train_loss_all=[]
for tea_num in range(n_teachers):
    print(f'start training number {tea_num} techer!')
    minloss =float ('inf')
    
    for epoch in range(num_epochs):
        train_loss = 0
        train_num=0
        teachers_models[tea_num].train()
        for b_x,b_y in teacher_data_loader[tea_num]:
            teacher_optimizers[tea_num].zero_grad()
            output=teachers_models[tea_num].forward(b_x.to(device))
            output=output.squeeze(-1)
            loss=loss_func(output,b_y.to(device))
            loss.backward()
            teacher_optimizers[tea_num].step()
            if loss< minloss:
                minloss = loss
                if os.path.exists(f'/home/ysy/ysy/Fed-ReKD-dirs/MLP_DP_SGD/teacher{tea_num}/best.pth'):
                    os.remove(f'/home/ysy/ysy/Fed-ReKD-dirs/MLP_DP_SGD/teacher{tea_num}/best.pth')
                torch.save(teachers_models[tea_num], f'/home/ysy/ysy/Fed-ReKD-dirs/MLP_DP_SGD/teacher{tea_num}/best.pth')
        if epoch%1== 0:
            print(f'Number:{tea_num}','epoch: {}/{} \t'.format(epoch+1,num_epochs),f'loss_tea:{loss}')
        train_loss_all.append(loss)
    print(f'finished training number {tea_num} techer!')

start training number 0 techer!
Number:0 epoch: 1/20 	 loss_tea:0.02730928361415863
Number:0 epoch: 2/20 	 loss_tea:0.019977986812591553
Number:0 epoch: 3/20 	 loss_tea:0.014993282966315746
Number:0 epoch: 4/20 	 loss_tea:0.016108613461256027
Number:0 epoch: 5/20 	 loss_tea:0.023659877479076385
Number:0 epoch: 6/20 	 loss_tea:0.018528813496232033
Number:0 epoch: 7/20 	 loss_tea:0.014356362633407116
Number:0 epoch: 8/20 	 loss_tea:0.017576448619365692
Number:0 epoch: 9/20 	 loss_tea:0.011448530480265617
Number:0 epoch: 10/20 	 loss_tea:0.019804460927844048
Number:0 epoch: 11/20 	 loss_tea:0.018628835678100586
Number:0 epoch: 12/20 	 loss_tea:0.023576494306325912
Number:0 epoch: 13/20 	 loss_tea:0.014419329352676868
Number:0 epoch: 14/20 	 loss_tea:0.013067604973912239
Number:0 epoch: 15/20 	 loss_tea:0.01613209955394268
Number:0 epoch: 16/20 	 loss_tea:0.01458014640957117
Number:0 epoch: 17/20 	 loss_tea:0.0138615183532238
Number:0 epoch: 18/20 	 loss_tea:0.014844628982245922
Number:0 e